In [1]:
import numpy
import scipy.special
import matplotlib.pyplot

%matplotlib inline

In [2]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hidden1nodes, hidden2nodes, outputnodes, learningrate):
        # 입력, 은닉, 출력계층의 노드갯수 설정
        self.inodes = inputnodes
        self.hnodes1 = hidden1nodes
        self.hnodes2 = hidden2nodes
        self.onodes = outputnodes
        
        # 가중치행렬 wih와 who
        self.wih1 = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.hnodes2, self.hnodes1))
        self.wh2o = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes2))
        
        self.lr = learningrate
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden2_errors = numpy.dot(self.wh2o.T, output_errors)
        hidden1_errors = numpy.dot(self.wh1h2.T, hidden2_errors)
        
        self.wh2o += self.lr*numpy.dot((output_errors*final_outputs*(1.0 - final_outputs)), numpy.transpose(hidden2_outputs))
        self.wh1h2 += self.lr*numpy.dot((hidden2_errors*hidden2_outputs*(1.0-hidden2_outputs)), numpy.transpose(hidden1_outputs))
        self.wih1 += self.lr*numpy.dot((hidden1_errors*hidden1_outputs*(1.0 - hidden1_outputs)), numpy.transpose(inputs))
        
        pass
    
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    


In [55]:
input_nodes = 784
hidden1_nodes = 108
# hidden layer 추가
hidden2_nodes = 108
output_nodes = 10
    
learning_rate = 0.2

# parameter에도 hidden layer 하나 더 추가
n = neuralNetwork(input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, learning_rate)

In [56]:
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [58]:
epochs = 15

for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
pass

In [64]:
%matplotlib inline
import imageio
import glob
import matplotlib.pyplot
image_list = []
for image_file_name in glob.glob('my_own_images/?.png'):
    correct_label = int(image_file_name[-5:-4])
    print("***correct_label:", correct_label)
    
    image_array = imageio.imread(image_file_name)
    
    #matplotlib.pyplot.imshow(image_array, cmap='Reds', interpolation='None')
    #matplotlib.pyplot.show()
    
    image_array = image_array[:,:,0]
    #print(image_array.shape)
    #print(image_array)
    image_array = numpy.reshape(image_array, (784,1))
    image_data = 255.0 - image_array
    image_data = (image_array / 255.0 * 0.99) + 0.01
    print(numpy.shape(image_data))
    image_list.append(image_data)
    #matplotlib.pyplot.imshow(image_array, cmap='Reds', interpolation='None')
    #matplotlib.pyplot.show()
print(len(image_list))

***correct_label: 1
(784, 1)
***correct_label: 2
(784, 1)
***correct_label: 3
(784, 1)
***correct_label: 4
(784, 1)
***correct_label: 5
(784, 1)
***correct_label: 6
(784, 1)
***correct_label: 7
(784, 1)
***correct_label: 8
(784, 1)
***correct_label: 9
(784, 1)
9


In [66]:
scorecard = []

#for record in test_data_list:
correct_label = [2,0,2,1,0,1,5,4,4]
i = 0
for record in image_list:
    #all_values = record.split(',')
    #numpy.reshape(record, (784,1))
    
    inputs = (numpy.asfarray(record) / 255.0 * 0.99) + 0.01

    outputs = n.query(inputs.T)
    label = numpy.argmax(outputs)
    print(label)
    if(label == correct_label[i]):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass
    i += 1
    


8
8
8
8
3
8
3
3
3


In [67]:
scorecard_array = numpy.asarray(scorecard)
print("Performance = ", scorecard_array.sum() / scorecard_array.size)

Performance =  0.0


In [69]:
print(scorecard)
scorecard_array=numpy.asarray(scorecard)
print("nodes =", hidden1_nodes, "rate =", learning_rate, "performance =", scorecard_array.sum() / scorecard_array.size)

[0, 0, 0, 0, 0, 0, 0, 0, 0]
nodes = 108 rate = 0.2 performance = 0.0
